In [1]:
# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
# You will also want to add some sample files to the "Files" (folder icon)
# on the left. When the agent asks you what to do, start with something
# simplie like "tell me what files are in this directory"
#
import os
with open("../../openai_token", "r") as file:
    openai_token = file.read().strip()
os.environ["OPENAI_API_KEY"] = openai_token

In [2]:
import json
import os
import sys
from litellm import completion
from typing import List, Dict

def extract_markdown_block(response: str, block_type: str = "json") -> str:
    """Extract code block from response"""

    if not '```' in response:
        return response

    code_block = response.split('```')[1].strip()

    if code_block.startswith(block_type):
        code_block = code_block[len(block_type):].strip()

    return code_block

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get a response."""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content.strip()

def parse_action(response: str) -> Dict:
    """Parse the LLM response into a structured action dictionary."""
    try:
        response = extract_markdown_block(response, "action")
        response_json = json.loads(response)
        if "tool_name" in response_json and "args" in response_json:
            return response_json
        else:
            return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
    except json.JSONDecodeError:
        return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

def list_files() -> List[str]:
    """List files in the current directory."""
    return os.listdir(".")

def read_file(file_name: str) -> str:
    """Read a file's contents."""
    try:
        with open(file_name, "r") as file:
            return file.read()
    except FileNotFoundError:
        return f"Error: {file_name} not found."
    except Exception as e:
        return f"Error: {str(e)}"
    
def summarize_file_content(file_content: str) -> str:
    """Summarize the content of a file using the LLM."""
    prompt = [
        {"role": "system", "content": "You are a helpful assistant that summarizes text."},
        {"role": "user", "content": f"Please summarize the following content:\n\n{file_content}"}
    ]
    response = generate_response(prompt)
    return response

# Define system instructions (Agent Rules)
agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:

```json
{
    "list_files": {
        "description": "Lists all files in the current directory.",
        "parameters": {}
    },
    "read_file": {
        "description": "Reads the content of a file.",
        "parameters": {
            "file_name": {
                "type": "string",
                "description": "The name of the file to read."
            }
        }
    },
    "terminate": {
        "description": "Ends the agent loop and provides a summary of the task.",
        "parameters": {
            "message": {
                "type": "string",
                "description": "Summary message to return to the user."
            }
        }
    },
    "summarize_file_content": {
        "description": "Summarizes the content of a file.",
        "parameters": {
            "file_content": {
                "type": "string",
                "description": "The content of the file to summarize."
            }
        }
    }
```

If a user asks about files, documents, or content, first list the files before reading them.

When you are done, terminate the conversation by using the "terminate" tool and I will provide the results to the user.

Important!!! Every response MUST have an action.
You must ALWAYS respond in this format:

<Stop and think step by step. Parameters map to args. Insert a rich description of your step by step thoughts here.>

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```"""
}]

# Initialize agent parameters
iterations = 0
max_iterations = 10

user_task = input("What would you like me to do? ")

memory = [{"role": "user", "content": user_task}]

# The Agent Loop
while iterations < max_iterations:
    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response: {response}")

    # 3. Parse response to determine action
    action = parse_action(response)
    result = "Action executed"

    if action["tool_name"] == "list_files":
        result = {"result": list_files()}
    elif action["tool_name"] == "read_file":
        result = {"result": read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error": action["args"]["message"]}
    elif action["tool_name"] == "summarize_file_content":
        file_content = action["args"]["file_content"]
        summary = summarize_file_content(file_content)
        result = {"summary": summary}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error": "Unknown action: " + action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.extend([
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    ])

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1


Agent thinking...
Agent response: First, I need to list the files in the current directory to identify which ones need summarizing. 

```action
{
    "tool_name": "list_files",
    "args": {}
}
```
Action result: {'result': ['bandits_file.txt', 'AgentWithTools.ipynb', 'churn_file.txt']}
Agent thinking...
Agent response: I will begin by reading the contents of each file one by one. Starting with "bandits_file.txt". 

```action
{
    "tool_name": "read_file",
    "args": {
        "file_name": "bandits_file.txt"
    }
}
```
Action result: {'result': 'Skip to content\nTowards Data Science\nPublish AI, ML & data-science insights to a global community of data professionals.\n\nSign in\nSubmit an Article\nLatest\nEditor’s Picks\nDeep Dives\nNewsletter\nWrite For TDS\nLinkedIn\nX\n\nToggle Search\nAn Overview of Contextual Bandits\nA dynamic approach to treatment personalization\n\nUgur Yildirim\nFeb 2, 2024\n23 min read\nShare\nOutline\nIntroduction\nWhen To Use Contextual Bandits 2.1. Conte